In [4]:
import d2lzh as d2l
from mxnet import nd
from mxnet.gluon import data as gdata
from IPython import display
from matplotlib import pyplot as plt
import time
import numpy as np

In [6]:
data = np.genfromtxt('D:/plantainz/OuO_DeepLearning/DataSet/airfoil_self_noise.dat',delimiter='\t')
data = (data - data.mean(axis = 0)) / data.std(axis = 0)
features,labels = np.array(data[:1500,:-1]),np.array(data[:1500,-1])

# Momentum
---

In [7]:
def init_momentum_states():
    v_w = nd.zeros((features.shape[1],1))
    v_b = nd.zeros(1)
    return (v_w,v_b)

In [9]:
def sgd_momentum(params,states,hyperparams):
    for p,v in zip(paramsms,states):
        v[:] = hyperparams['momentum'] * v + hyperparams['lr'] * p.grad
        p[:] -= v 

# Adagrad
---

In [10]:
def adagrad_2d(x1,x2,s1,s2):
    g1,g2,eps = 0.2*x1,4*x2,1e-6
    s1 += g1**2
    s2 += g2**2
    
    x1 -= eta / math.sqrt(s1 + eps)*g1
    x2 -= eta / math.sqrt(s2 + eps)*g2
    return x1,x2,s1,s2

In [ ]:
def f_2d(x1,x2): # 给Adagrad和下方RMSProp的示例函数
    return 0.1*x1**2 + 2*x2**2

# RMSProp
---

In [ ]:
def rmsprop_2d(x1,x2,s1,s2):
    g1,g2,eps = 0.2*x1,4*x2,1e-6
    s1 = gamma * s1+(1-gamma)*g1**2
    s2 = gamma * s2+(1-gamma)*g2**2
    x1 -= eta / math.sqrt(s1 + eps)*g1
    x2 -= eta / math.sqrt(s2 + eps)*g2
    
    return x1,x2,s1,s2

In [11]:
def init_rmsprop_states():
    s_w = nd.zeros((features.shape[1],1))
    s_b = nd.zeros(1)
    return (s_w,s_b)

In [12]:
def rms_prop(params,states,hyperparams):
    gamma,eps = hyperparams['gamma'],1e-6
    for p,s in zip(params,states):
        s[:] = gamma*s + (1-gamma)*p.grad.square()
        

SyntaxError: invalid syntax (<ipython-input-12-481ea4a169c3>, line 1)